In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
import json
import os
from sklearn.decomposition import PCA

# Load model InceptionV3
model = InceptionV3(weights='imagenet', include_top=False)  # exclude the top dense layers

# Function to extract features from an image
def extract_features(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    img_preprocessed = preprocess_input(img_array_expanded_dims)

    # Extract features from the model
    features = model.predict(img_preprocessed)
    flattened_features = features.flatten()  # Flatten the features into a vector
    return flattened_features

# Function to reduce dimensionality using PCA
def reduce_dimensionality(features, pca):
    reduced_features = pca.transform(features)
    return reduced_features

# Function to save extracted features with reduced dimensionality to JSON file
def save_features_to_json(base_folder_path, pca):
    features_list = []
    # Duyệt qua tất cả các tệp trong base_folder_path
    for filename in os.listdir(base_folder_path):
        img_path = os.path.join(base_folder_path, filename)
        if os.path.isfile(img_path) and (filename.endswith(".jpg") or filename.endswith(".png")):
            features = extract_features(img_path)
            if len(features) > 0:
              features_reshaped = np.array(features).reshape(1, -1)
              reduced_features = reduce_dimensionality(features_reshaped, pca)
              features_list.append({'features': reduced_features.tolist()[0], 'path': img_path})


    # Lưu vào tệp JSON
    with open('features.json', 'w') as f:
        json.dump(features_list, f, indent=4)


# Path to the directory containing images
folder_path = '/content/drive/MyDrive/Master/Multimedia/fruit'

# Number of dimensions after dimensionality reduction
pca_dim = 50

# Fit PCA with data
features = []
# Duyệt qua tất cả các tệp trong thư mục folder_path
for filename in os.listdir(folder_path):
    img_path = os.path.join(folder_path, filename)
    if os.path.isfile(img_path) and (filename.endswith(".jpg") or filename.endswith(".png")):
        features.append(extract_features(img_path))
features = np.array(features)

pca = PCA(n_components=pca_dim)
pca.fit(features)

# Save extracted features with reduced dimensionality to JSON file
save_features_to_json(folder_path, pca)


1/1 [==============================] - 0s 38ms/step


In [ ]:
import joblib

joblib.dump(pca, 'pca.pkl')

['pca.pkl']

In [ ]:
import shutil

# Đường dẫn của file cần sao lưu trong Colab
source_path = '/content/features.json'

# Đường dẫn của thư mục đích trong Google Drive
destination_path = '/content/drive/My Drive/Master/Multimedia//'

# Sao chép file từ đường dẫn nguồn đến đường dẫn đích
shutil.copy(source_path, destination_path)

print("File đã được sao lưu vào Google Drive!")


File đã được sao lưu vào Google Drive!
